In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'svg'
config = {'font.family': 'Times New Roman',
          'font.size': 15,
          'xtick.direction': 'in',
          'ytick.direction': 'in',
          'mathtext.fontset': 'stix',
         }
plt.rcParams.update(config)

scenario = "B"

results = np.load(f"./output/{scenario}/results.npy").reshape(181, 30, 100, 400) # 时刻 种类
n = np.load(f"./output/{scenario}/results_porosity.npy").reshape(181, 100, 400)
K = np.load(f"./output/{scenario}/results_K.npy").reshape(181, 100, 400)
K_log = np.log(K)

print(results.shape)
print(n.shape)
print(K.shape)
# print(heads.shape)

# ['0 K' '1 Na' '2 Ca' '3 Mg' '4 Li' '5 Cl' '6 S(6)' '7 C(4)' '8 Halite'
#  '9 Carnallite' '10 Polyhalite' '11 Sylvite' '12 Gypsum' '13 Calcite' '14 Dolomite'
#  '15 d_Halite' '16 d_Carnallite' '17 d_Polyhalite' '18 d_Sylvite' '19 d_Gypsum' '20 d_Calcite' '21 d_Dolomite'
#  '22 density(kg/m3)'
#  '23 SI_Halite' '24 SI_Carnallite' '25 SI_Polyhalite' '26 SI_Sylvite' '27 SI_Gypsum' '28 SI_Calcite' '29 SI_Dolomite']

(181, 30, 100, 400)
(181, 100, 400)
(181, 100, 400)


In [3]:
import numpy as np
import pandas as pd

# --- 假设你的数据和变量已准备好 ---
# results: (61, 30, 100, 400)
# ny, nx = 100, 400
# time_steps_analyze: (60,) - 对应变化步的时间点 (e.g., from np.linspace)
# analysis_indices: [11, 35, 59] - 前面计算得到的年末 1, 3, 5 的索引
# analysis_years: [1, 3, 5]

# --- 定义矿物索引和名称 ---
mineral_indices_map = {
    'Halite': 8,
    'Carnallite': 9,
    'Polyhalite': 10,
    'Sylvite': 11,
    'Gypsum': 12,
    'Calcite': 13,
    'Dolomite': 14
}
mineral_names_list = list(mineral_indices_map.keys())
mineral_indices_list = list(mineral_indices_map.values())

# --- 初始化存储结果的字典 ---
efficiency_results = {} # 存储转换效率
extent_results = {} # 存储波及效率


# --- 计算初始矿物总量 (只需计算一次) ---
initial_total_volumes = {}
print("Calculating initial total volumes for minerals...")
for name, index in mineral_indices_map.items():
    vol = np.sum(results[0, index, :, :])
    initial_total_volumes[name] = vol
    print(f"  Initial Total Volume - {name}: {vol:.4e}")

# --- 容忍度，用于判断是否溶解 ---
dissolution_tolerance = 1e-9

target_days = [180, 1095, 1825] # 大约是年 1, 3, 5 结束
analysis_indices = []
analysis_years = [1, 3, 5]
n_times, ny, nx = K.shape
time_steps_days = np.linspace(0, 1825, n_times)
time_steps_analyze = time_steps_days[1:] # 我们分析变化，从第二个时间点开始

print("确定分析时间点对应的数组索引...")
for i, day in enumerate(target_days):
    # 找到 time_steps_analyze 中最接近目标日期的索引
    # time_steps_analyze 是从第一个变化步开始的时间点 (对应 delta_K 等数组的索引 0)
    idx = np.abs(time_steps_analyze - day).argmin()
    analysis_indices.append(idx)
    print(f"  Year {analysis_years[i]} (Target Day ~{day}): Closest time step is Day {time_steps_analyze[idx]:.1f} at index {idx}")

# --- 循环计算每个时间点的指标 ---
print("\nCalculating efficiencies for target years...")
for i, analysis_time_index in enumerate(analysis_indices):
    current_year = analysis_years[i]
    time_label = f'Year_{current_year}' # 用于字典的键和列名

    # -- 计算固液转换效率 --
    current_efficiencies = {}
    for name, index in mineral_indices_map.items():
        initial_vol = initial_total_volumes[name]
        # 使用 analysis_time_index + 1 访问 results 数组，因为 results[0] 是初始状态
        final_vol = np.sum(results[analysis_time_index + 1, index, :, :])
        net_dissolved = initial_vol - final_vol

        if initial_vol > dissolution_tolerance: # 避免除以零或非常小的数
            efficiency = (net_dissolved / initial_vol) * 100
        elif net_dissolved < -dissolution_tolerance: # 初始为0但有沉淀
             efficiency = -np.inf # 或标记为 "Net Precipitation from zero"
        else: # 初始为0且无明显变化或溶解
            efficiency = 0.0 # 或标记为 "N/A"

        current_efficiencies[name] = efficiency
    efficiency_results[time_label] = current_efficiencies

    # -- 计算溶矿波及效率 --
    overall_dissolution_mask = np.zeros((ny, nx), dtype=bool)
    for index in mineral_indices_list:
        delta_mineral = results[analysis_time_index + 1, index, :, :] - results[0, index, :, :]
        dissolved_mask_m = (delta_mineral < -dissolution_tolerance)
        overall_dissolution_mask = np.logical_or(overall_dissolution_mask, dissolved_mask_m)

    num_dissolved_cells = np.sum(overall_dissolution_mask)
    total_cells = ny * nx
    dissolution_extent = (num_dissolved_cells / total_cells) * 100
    extent_results[time_label] = dissolution_extent

    print(f"  Finished calculations for Year {current_year}")

# --- 3. 格式化输出为表格 ---

# 创建转换效率表格
efficiency_df = pd.DataFrame(efficiency_results)
efficiency_df.index.name = 'Mineral'
# 添加一个总计行，可能意义不大，但可以考虑
# efficiency_df.loc['Total Efficiency (Simple Sum)', :] = efficiency_df.sum(axis=0)

# 创建波及效率表格 (只有一行)
extent_df = pd.DataFrame([extent_results], index=['Overall Dissolution Extent (%)'])

# 合并两个表格 (可选，如果想放一起)
# combined_df = pd.concat([efficiency_df, extent_df])

print("\n===== Mineral Transformation Efficiency (%) =====")
print(" (Positive value indicates net dissolution relative to initial amount)")
print(" (Negative value indicates net precipitation relative to initial amount)")
print(efficiency_df.to_string(float_format="%.2f")) # 格式化输出

print("\n===== Overall Dissolution Extent (%) =====")
print(" (Percentage of grid cells experiencing net dissolution of at least one mineral)")
print(extent_df.to_string(float_format="%.2f"))

# 如果想合并输出:
# print("\n===== Combined Efficiency Metrics =====")
# print(combined_df.to_string(float_format="%.2f"))

Calculating initial total volumes for minerals...
  Initial Total Volume - Halite: 3.0800e+06
  Initial Total Volume - Carnallite: 7.1286e+04
  Initial Total Volume - Polyhalite: 1.0800e+05
  Initial Total Volume - Sylvite: 2.6379e+04
  Initial Total Volume - Gypsum: 1.7560e+05
  Initial Total Volume - Calcite: 0.0000e+00
  Initial Total Volume - Dolomite: 0.0000e+00
确定分析时间点对应的数组索引...
  Year 1 (Target Day ~180): Closest time step is Day 182.5 at index 17
  Year 3 (Target Day ~1095): Closest time step is Day 1095.0 at index 107
  Year 5 (Target Day ~1825): Closest time step is Day 1825.0 at index 179

Calculating efficiencies for target years...
  Finished calculations for Year 1
  Finished calculations for Year 3
  Finished calculations for Year 5

===== Mineral Transformation Efficiency (%) =====
 (Positive value indicates net dissolution relative to initial amount)
 (Negative value indicates net precipitation relative to initial amount)
            Year_1  Year_3  Year_5
Mineral     

In [4]:
import numpy as np
import pandas as pd

# --- 假设你的数据和变量已准备好 ---
# results: (61, 30, 100, 400)
# ny, nx = 100, 400
# time_steps_analyze: (60,)
# analysis_indices: [11, 35, 59]
# analysis_years: [1, 3, 5]
# mineral_indices_map = {...}
# mineral_names_list = list(mineral_indices_map.keys())
# mineral_indices_list = list(mineral_indices_map.values())

# --- 定义溶解阈值 (绝对体积分数变化) ---
# !!! 这个值需要根据你的模拟结果和物理意义来调整 !!!
# 例如，-1e-1 表示体积分数减少超过 10% 才算显著溶解
absolute_dissolution_threshold = 0.15
print(f"Using Absolute Dissolution Threshold: Volume fraction decrease > {absolute_dissolution_threshold:.1e}")

# --- 初始化存储结果的字典 ---
mineral_extent_results = {} # 存储单矿物波及效率
overall_extent_results = {} # 存储总体波及效率
# (固液转换效率部分代码不变，可以保留之前的 calculation 和 results)
# efficiency_results = {} # 假设这个也重新计算或已存在

# --- (如果需要，重新计算固液转换效率的代码部分 - 和上次一样) ---
initial_total_volumes = {}
dissolution_tolerance = 1e-9 # 用于转换效率计算中的除零判断
efficiency_results = {}
print("\nCalculating Transformation Efficiency (same as before)...")
for name, index in mineral_indices_map.items():
    initial_total_volumes[name] = np.sum(results[0, index, :, :])
for i, analysis_time_index in enumerate(analysis_indices):
    current_year = analysis_years[i]
    time_label = f'Year_{current_year}'
    current_efficiencies = {}
    for name, index in mineral_indices_map.items():
        initial_vol = initial_total_volumes[name]
        final_vol = np.sum(results[analysis_time_index + 1, index, :, :])
        net_dissolved = initial_vol - final_vol
        if initial_vol > dissolution_tolerance:
            efficiency = (net_dissolved / initial_vol) * 100
        elif net_dissolved < -dissolution_tolerance:
             efficiency = -np.inf
        else:
            efficiency = 0.0
        current_efficiencies[name] = efficiency
    efficiency_results[time_label] = current_efficiencies
print("Transformation Efficiency calculated.")


# --- 循环计算每个时间点的波及效率 ---
print("\nCalculating Mineral-Specific and Overall Dissolution Extent with Threshold...")
for i, analysis_time_index in enumerate(analysis_indices):
    current_year = analysis_years[i]
    time_label = f'Year_{current_year}' # 用于字典的键和列名

    current_mineral_extents = {}
    overall_significant_dissolution_mask = np.zeros((ny, nx), dtype=bool)

    for name, index in mineral_indices_map.items():
        # 计算净变化量
        delta_mineral = results[analysis_time_index + 1, index, :, :] - results[0, index, :, :]

        # 标记显著溶解的网格
        significant_dissolved_mask_m = (delta_mineral < -absolute_dissolution_threshold)

        # 计算该矿物的波及效率
        num_significant_dissolved_cells_m = np.sum(significant_dissolved_mask_m)
        total_cells = ny * nx
        mineral_extent = (num_significant_dissolved_cells_m / total_cells) * 100
        current_mineral_extents[name] = mineral_extent

        # 更新总体掩码
        overall_significant_dissolution_mask = np.logical_or(overall_significant_dissolution_mask, significant_dissolved_mask_m)

    # 计算总体显著波及效率
    num_overall_significant_dissolved_cells = np.sum(overall_significant_dissolution_mask)
    overall_extent = (num_overall_significant_dissolved_cells / total_cells) * 100

    # 存储结果
    mineral_extent_results[time_label] = current_mineral_extents
    overall_extent_results[time_label] = overall_extent

    print(f"  Finished extent calculations for Year {current_year}")

# --- 格式化输出为表格 ---

# 创建固液转换效率表格 (如果需要)
efficiency_df = pd.DataFrame(efficiency_results)
efficiency_df.index.name = 'Mineral'

# 创建单矿物溶解波及效率表格
mineral_extent_df = pd.DataFrame(mineral_extent_results)
mineral_extent_df.index.name = 'Mineral'

# 创建总体显著溶矿波及效率表格
overall_extent_df = pd.DataFrame([overall_extent_results], index=['Overall Significant Dissolution Extent (%)'])

# --- 打印表格 ---
print("\n===== Mineral Transformation Efficiency (%) =====")
print(" (Net Dissolution (+) or Precipitation (-) relative to initial total volume)")
print(efficiency_df.to_string(float_format="%.2f"))

print(f"\n===== Mineral-Specific Dissolution Extent (%) =====")
print(f" (Percentage of cells where mineral dissolved by > {absolute_dissolution_threshold:.1e} vol. fraction)")
print(mineral_extent_df.to_string(float_format="%.2f"))

print(f"\n===== Overall Significant Dissolution Extent (%) =====")
print(f" (Percentage of cells where at least one mineral dissolved by > {absolute_dissolution_threshold:.1e} vol. fraction)")
print(overall_extent_df.to_string(float_format="%.2f"))

Using Absolute Dissolution Threshold: Volume fraction decrease > 1.5e-01

Calculating Transformation Efficiency (same as before)...
Transformation Efficiency calculated.

Calculating Mineral-Specific and Overall Dissolution Extent with Threshold...
  Finished extent calculations for Year 1
  Finished extent calculations for Year 3
  Finished extent calculations for Year 5

===== Mineral Transformation Efficiency (%) =====
 (Net Dissolution (+) or Precipitation (-) relative to initial total volume)
            Year_1  Year_3  Year_5
Mineral                           
Halite       -0.08   -0.46   -0.77
Carnallite    4.24   23.57   40.24
Polyhalite    0.09    0.71    1.17
Sylvite      -6.25  -34.16  -59.84
Gypsum       -0.11   -0.80   -1.31
Calcite       0.00    0.00    0.00
Dolomite      0.00    0.00    0.00

===== Mineral-Specific Dissolution Extent (%) =====
 (Percentage of cells where mineral dissolved by > 1.5e-01 vol. fraction)
            Year_1  Year_3  Year_5
Mineral             